In [1]:
stones = "92 0 286041 8034 34394 795 8 2051489"
stones_list = stones.split()

## Part 1

In [2]:
# Evolve the stones over 25 blinks
for i in range(25):

    new_stones = []
    for stone in stones_list:
        
        if stone == "0":
            new_stones.append("1")
            
        elif len(stone) % 2 == 0:
            mid = len(stone) // 2
            left = stone[:mid].lstrip("0") or "0"
            right = stone[mid:].lstrip("0") or "0"
            new_stones.extend([left, right])
            
        else:
            new_stones.append(str(int(stone) * 2024))
            
    stones_list = new_stones

print(len(stones_list), stones_list[:20])

239714 ['3', '2', '7', '7', '2', '6', '16192', '4048', '1', '4048', '8096', '3', '6', '8', '6', '9', '1', '8', '4', '2']
